### Overview

In [1]:
## The objective of this analysis is the evaluation of the sales of Electric Vehicles (EV's) in the state of Washington.
## Sales sensitiviy analyses are done along several dimensions including:
    ## Battery Range
    ## Vehicle Price
    ## Population
    ## Distance of Travel (Surface Area of Vehicle Home Location)
    ## Availability of Charging Stations outside the home
    
## More focused analysis will be done on the top five EV makes that currently hold over 80% of the market share

## Recommendations will be made to guide further R&D and Product Development decisions regarding features vs range.

### Sources of Data

In [2]:
## Electric Vehicle dataset was obtained at data.gov: https://catalog.data.gov/dataset/electric-vehicle-population-data
## Electric Vehicle Charging Stations data: https://chargehub.com/en/countries/united-states/washington/seattle.html
## Electric Rates Per City in Washington State: https://www.electricitylocal.com/states/washington
## Washington State City and County Demographics: https://www.washington-demographics.com/counties_by_population

### Electric Vehicles Categories

In [3]:
## There are two types of electric vehicles captured in the dataset from data.gov.  They are:
    ## Battery Electric Vehicles - entirely fueled by an battery banks
    ## Plug-in Hybrid Electric Vehicle - fueled by batteries and an additional fuel (gasoline, diesel, ethanol)
    
## This project will focused on BEV's although some comparisons will be made with PHEV data for context definition

### Dependencies and Set up

In [4]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

pd.options.display.float_format = "{:,.3f}".format

In [5]:
# Reading the datafiles
EV_raw_data = pd.read_csv("Wash_State_EV_PHEV_Data.csv")
County_demo_data = pd.read_csv("Wash_State_County_Demo_Data.csv")

In [13]:
EV_raw_data = pd.DataFrame(EV_raw_data)
EV_raw_data.head()

,City,County,ZIP Code,VIN (1-10),Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,AUBURN,King,98001,5YJYGDEEXL,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,315.000,"42,190.000",30.000,"114,288,233.000",POINT (-122.266685 47.308313),nan,nan,nan
1,AUBURN,King,98001,WVWKR7AU1K,2019,VOLKSWAGEN,E-GOLF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,125.000,"32,790.000",47.000,"331,142,717.000",POINT (-122.266685 47.308313),nan,nan,nan
2,AUBURN,King,98001,5YJYGDEE1L,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,315.000,"42,190.000",47.000,"112,961,761.000",POINT (-122.266685 47.308313),nan,nan,nan
3,AUBURN,King,98001,1N4AZ1BP2L,2020,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,149.000,"31,620.000",30.000,"132,473,834.000",POINT (-122.266685 47.308313),nan,nan,nan
4,AUBURN,King,98001,5YJSA1H11E,2014,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.000,"69,900.000",30.000,"2,267,016.000",POINT (-122.266685 47.308313),nan,nan,nan


In [7]:
# Release unneeded columns from EV_raw_data

In [8]:
EV_raw_data.drop(["VIN","Legislative_District","DOL_ID","Vehicle_Location"],axis=1,inplace=True)

KeyError: "['VIN' 'Legislative_District' 'DOL_ID' 'Vehicle_Location'] not found in axis"

In [9]:
EV_raw_data

,City,County,ZIP Code,VIN (1-10),Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,AUBURN,King,98001,5YJYGDEEXL,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,315.000,"42,190.000",30.000,"114,288,233.000",POINT (-122.266685 47.308313),nan,nan,nan
1,AUBURN,King,98001,WVWKR7AU1K,2019,VOLKSWAGEN,E-GOLF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,125.000,"32,790.000",47.000,"331,142,717.000",POINT (-122.266685 47.308313),nan,nan,nan
2,AUBURN,King,98001,5YJYGDEE1L,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,315.000,"42,190.000",47.000,"112,961,761.000",POINT (-122.266685 47.308313),nan,nan,nan
3,AUBURN,King,98001,1N4AZ1BP2L,2020,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,149.000,"31,620.000",30.000,"132,473,834.000",POINT (-122.266685 47.308313),nan,nan,nan
4,AUBURN,King,98001,5YJSA1H11E,2014,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.000,"69,900.000",30.000,"2,267,016.000",POINT (-122.266685 47.308313),nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64427,CLARKSTON,Asotin,99403,KMHE34L18G,2016,HYUNDAI,SONATA PLUG-IN HYBRID,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,27.000,"34,600.000",9.000,"192,386,785.000",POINT (-117.238638 46.260282),nan,nan,nan
64428,CLARKSTON,Asotin,99403,JTDKN3DP8C,2012,TOYOTA,PRIUS,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,6.000,"32,000.000",9.000,"120,597,046.000",POINT (-117.238638 46.260282),nan,nan,nan
64429,CLARKSTON,Asotin,99403,JTDKARFP8L,2020,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25.000,"27,600.000",9.000,"103,464,763.000",POINT (-117.238638 46.260282),nan,nan,nan
64430,CLARKSTON,Asotin,99403,1G1RD6E4XE,2014,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38.000,"34,185.000",9.000,"218,558,910.000",POINT (-117.238638 46.260282),nan,nan,nan


In [10]:
# Compare Cars by Type of Electric Vehicle
    # BoxPlot of Range
    # BoxPlot of Price

In [11]:
# BoxPlot of Driving Range Based on Vehicle Type

red_square = dict(markerfacecolor = 'r', marker ='s')

EV_raw_data.boxplot("Electric_Range", by="EV_Type", figsize = (20,10), notch = True, flierprops = red_square)

plt.title("Driving Ranges by Type of Electric Vehicle",fontdict ={"fontsize":24})
plt.xlabel("Electric Vehicle Type", fontdict={"fontsize":22})
plt.ylabel("Range in miles", fontdict={"fontsize":22})
plt.tight_layout(pad = 2.0, w_pad = 1.5, h_pad = 2.0)

plt.show()


KeyError: 'EV_Type'

In [12]:
# BoxPlot of Price Range Based on Vehicle Type

blue_square = dict(markerfacecolor = 'b', marker ='s')

EV_raw_data.boxplot("Base_MSRP", by="EV_Type", figsize = (20,10), notch = True, flierprops = blue_square)

plt.title("MSRP by Type of Electric Vehicle",fontdict ={"fontsize":24})
plt.xlabel("Electric Vehicle Type", fontdict={"fontsize":22})
plt.ylabel("Price ($)", fontdict={"fontsize":22})
plt.tight_layout(pad = 2.0, w_pad = 1.5, h_pad = 2.0)

plt.show()


KeyError: 'EV_Type'

### Separate Battery Electric Vehicles and Plug-in Hybrid Electric Vehicles into two datasets

In [53]:
BEV_df = EV_raw_data[EV_raw_data.EV_Type== "Battery Electric Vehicle (BEV)"]
BEV_df

,City,County,ZIP Code,Model_Year,Make,Model,EV_Type,Electric_Range,Base_MSRP
0,AUBURN,King,98001,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),315.000,"42,190.000"
1,AUBURN,King,98001,2019,VOLKSWAGEN,E-GOLF,Battery Electric Vehicle (BEV),125.000,"32,790.000"
2,AUBURN,King,98001,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),315.000,"42,190.000"
3,AUBURN,King,98001,2020,NISSAN,LEAF,Battery Electric Vehicle (BEV),149.000,"31,620.000"
4,AUBURN,King,98001,2014,TESLA,MODEL S,Battery Electric Vehicle (BEV),208.000,"69,900.000"
...,...,...,...,...,...,...,...,...,...
46000,CLARKSTON,Asotin,99403,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),75.000,"28,800.000"
46001,CLARKSTON,Asotin,99403,2018,TESLA,MODEL X,Battery Electric Vehicle (BEV),238.000,"79,500.000"
46002,CLARKSTON,Asotin,99403,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),322.000,"39,190.000"
46003,CLARKSTON,Asotin,99403,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),151.000,"29,990.000"


In [54]:
PHEV_df = EV_raw_data[EV_raw_data.EV_Type== "Plug-in Hybrid Electric Vehicle (PHEV)"]
PHEV_df

,City,County,ZIP Code,Model_Year,Make,Model,EV_Type,Electric_Range,Base_MSRP
46005,BELLEVUE,King,98015,2017,BMW,X5 XDRIVE40E,Plug-in Hybrid Electric Vehicle (PHEV),14.000,"63,200.000"
46006,PAWCATUCK,New London,6379,2014,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),38.000,"34,185.000"
46007,HADDONFIELD,Camden,8033,2019,HYUNDAI,IONIQ PLUG-IN HYBRID,Plug-in Hybrid Electric Vehicle (PHEV),29.000,"25,350.000"
46008,SILVER SPRING,Montgomery,20906,2015,FORD,FUSION ENERGI,Plug-in Hybrid Electric Vehicle (PHEV),19.000,"34,800.000"
46009,ANNAPOLIS,Anne Arundel,21401,2012,TOYOTA,PRIUS,Plug-in Hybrid Electric Vehicle (PHEV),6.000,"32,000.000"
...,...,...,...,...,...,...,...,...,...
64427,CLARKSTON,Asotin,99403,2016,HYUNDAI,SONATA PLUG-IN HYBRID,Plug-in Hybrid Electric Vehicle (PHEV),27.000,"34,600.000"
64428,CLARKSTON,Asotin,99403,2012,TOYOTA,PRIUS,Plug-in Hybrid Electric Vehicle (PHEV),6.000,"32,000.000"
64429,CLARKSTON,Asotin,99403,2020,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),25.000,"27,600.000"
64430,CLARKSTON,Asotin,99403,2014,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),38.000,"34,185.000"


## Breaking Down BEV's Per County

In [67]:
BEV_Per_County_df = BEV_df.groupby("County").Make.count()
BEV_Per_County_df = pd.DataFrame(BEV_Per_County_df)
BEV_Per_County_df.set_index(drop=True)
BEV_Per_County_df

TypeError: set_index() missing 1 required positional argument: 'keys'

AttributeError: 'Series' object has no attribute 'columns'

In [ ]:
# This program examines the eficacy in treating SCC tumors using several drug treatments including Capomulin
# This analysis was completed by F. A. Barillas

In [ ]:
# Checking the number of mice in the clean DataFrame.
clean_study_results.duplicated(subset=['Mouse_ID','Timepoint']).sum()

In [ ]:
len(clean_study_results['Mouse_ID'])

In [ ]:
# Merge Mouse Data with Study Data
clean_study_results = pd.merge(mouse_metadata, clean_study_results, how = "left", on = "Mouse_ID")
clean_study_results

# CLEAN STUDY RESULTS DATAFRAME WITHOUT DUPLICATES

In [ ]:
clean_study_results.rename(columns={'Weight_(g)':'Weight_g', 'Tumor_Volume_(mm3)':'Tumor_Volume_mm3'}, inplace =True)
clean_study_results

In [ ]:
male_mice = (clean_study_results.Sex =="Male").sum()
male_mice


In [ ]:
female_mice = (clean_study_results.Sex =="Female").sum()
female_mice


## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for 
# each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for 
# each regimen

In [ ]:
# Undertanding Trials Per Regimen
Trials_Per_Regimen = clean_study_results.groupby('Drug_Regimen').Drug_Regimen.count()


In [ ]:
Mean_Tumor_Volume = clean_study_results.groupby('Drug_Regimen').Tumor_Volume_mm3.mean()


In [ ]:
Median_Tumor_Volume = clean_study_results.groupby('Drug_Regimen').Tumor_Volume_mm3.median()


In [ ]:
Variance_Tumor_Volume = clean_study_results.groupby('Drug_Regimen').Tumor_Volume_mm3.var()


In [ ]:
StDev_Tumor_Volume = clean_study_results.groupby('Drug_Regimen').Tumor_Volume_mm3.std()


In [ ]:
SEM_Tumor_Volume = clean_study_results.groupby('Drug_Regimen').Tumor_Volume_mm3.sem()


In [ ]:
# Using the aggregation method, produce the same summary statistics in a single line
summary_stats_df=pd.concat([Trials_Per_Regimen, Mean_Tumor_Volume, Median_Tumor_Volume, Variance_Tumor_Volume, StDev_Tumor_Volume, SEM_Tumor_Volume], axis =1)
summary_stats_df.columns=['Trials','Average','Median','Variance','StDev','SEM']              


In [ ]:
summary_stats_df.reset_index(inplace = True)

# SUMMARY STATISTICS BY DRUG REGIMEN

In [ ]:
summary_stats_df

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.
summary_stats_df.sort_values('Trials', inplace = True, ascending = False)

x_axis = summary_stats_df['Drug_Regimen']
y_values=summary_stats_df['Trials']

tick_locations = [value for value in x_axis]

fig1 = plt.figure(figsize =(12,6))

plt.bar(x_axis, y_values, color = 'b', alpha = 0.75, align = "center", width = 0.5)
plt.title("Trials Per Drug Regimen")
plt.xlabel("Drug Regimen")
plt.ylabel("Number of Trials")

plt.xlim(-0.5, len(x_axis)-0.5)

plt.grid(color='k', alpha = 0.15)
plt.tight_layout(pad=0.8, w_pad=1.4, h_pad=1.0)


In [ ]:
categories = [male_mice,female_mice]
categories

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
fig2 = plt.figure()
fig2.suptitle('Mice Gender Split', fontsize = 14, fontweight = 'bold')


labels = ["Male Mice", "Female Mice" ]
categories = [male_mice, female_mice]
colors = ["y","c"]
explode = (0.1,0)

plt.pie(categories, explode, labels, colors, autopct='%1.1f%%', shadow= True, startangle = 45)
plt.axis("Equal")
plt.tight_layout()
plt.show()


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

Capomulin_df = clean_study_results[clean_study_results.Drug_Regimen == "Capomulin"]
Ramicane_df = clean_study_results[clean_study_results.Drug_Regimen == "Ramicane"]
Infubinol_df = clean_study_results[clean_study_results.Drug_Regimen == "Infubinol"]
Ceftamin_df = clean_study_results[clean_study_results.Drug_Regimen == "Ceftamin"]

In [ ]:
# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
